In [1]:
# import ctadata as cd
# import ctaplots as cplt
import matplotlib.pyplot as plt
import os
from ctaDataWrapper import PCalibRun
import numpy as np

/Users/thomasvuillaume/anaconda/lib/python3.5/site-packages/matplotlib/font_manager.py:280: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [2]:
%config InlineBackend.figure_format = 'retina'
np.set_printoptions(precision=3)
from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

## Load calibrated prun file

You may find a small one in share

In [22]:
pruncalibfilename = '../share/PRUN_CALIB/gamma_20deg_0deg_run337___cta-prod3-merged_desert-2150m--subarray-3-nosct.pcalibRun'

In [23]:
pr = PCalibRun()
pr.load(pruncalibfilename)

In [24]:
hr = pr.header

In [25]:
tel = pr.tabTelescope[0]
event = tel.tabTelEvent[0]

In [26]:
event.tabPixel

array([-0.47 ,  0.136,  0.953, ...,  1.786,  3.303,  0.745], dtype=float32)

In [27]:
np.array([event.tabPixel for tel in pr.tabTelescope for event in tel.tabTelEvent if tel.telescopeType == 0])

array([[-0.47 ,  0.136,  0.953, ...,  1.786,  3.303,  0.745],
       [-1.566, -0.793, -0.073, ..., -0.441,  2.782,  0.587],
       [-1.119,  2.9  ,  1.71 , ...,  2.041, -0.113,  0.248],
       ..., 
       [-1.294,  0.85 , -0.242, ..., -0.161, -1.463,  0.   ],
       [ 1.861, -0.604,  1.43 , ..., -1.677, -1.681,  1.309],
       [-1.43 ,  1.409, -1.408, ...,  0.551, -0.851,  0.266]], dtype=float32)